In [ ]:
#High exposure vs low exposure captains (pre vs post)

In [ ]:
#captain id - HE/LE - APR,

In [42]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import seaborn as sns
sns.set()
import gc
gc.collect()
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
from IPython.display import clear_output
#set_matplotlib_formats('retina')
#from tqdm import tqdm
#tqdm.pandas()
import numpy as np
from pyhive import presto
from datetime import datetime, timedelta
#from bson import ObjectId
#from functools import reduce
#from sklearn.cluster import KMeans
#from pymongo import MongoClient
import glob
import warnings
warnings.filterwarnings('ignore')
from datetime import date
import json
#import re
#import dtale
#from h3 import h3
# import pandasql as ps

In [43]:
metabase_connection = presto.connect(
        #host='bi-trino-2.serving.data.production.internal',
        #host='bi-trino.serving.data.production.internal',
        #host='bi-presto.serving.data.production.internal',
        #host='prime-trino.serving.data.production.internal',
        #host="presto.processing.yoda.run",
        host='presto-gateway.processing.data.production.internal',
        port=80,
        protocol='http',
        catalog ='hive',
        username='praveen.u@rapido.bike')

In [ ]:
#stratify based on - previous aprs, previous rides, number of pings, stress, ratio of pings in test and control

In [ ]:
#get post perido, apr, no of rides , in test and control hexs

In [ ]:
captain id, apr_test2,apr_ctrl2,apr_ctrl3,apr_ctrl1,total_pings_test,total_pings_control,total_pings

In [52]:
#pre APR
query="""

with order_filtering as        
    (
    select * from
        (
        select distinct yyyymmdd,order_id
        from 
            orders.order_logs_snapshot
            --where yyyymmdd = cast({dt} as varchar)
            where yyyymmdd >= '20230704' and yyyymmdd <= '20230722'
            and service_obj_service_name = 'Auto'
            and (spd_fraud_flag = false OR spd_fraud_flag IS NULL)
            and  city_name in ('Hyderabad','Pune') 
           )
        ),
           

    t1 as (
    
        select yyyymmdd,test_ctrl,time_period,data_riderId,city_name,
            count(distinct data_orderid) as Gross_orders,
            count(distinct case when event_type = 'rider_busy' then unique_ping_level_id end) as riderBusy_pings,
            count(distinct case when event_type = 'rider_rejected' then unique_ping_level_id end) as riderreject_Pings,
            count(distinct case when event_type = 'rider_accepted' then unique_ping_level_id end) as accepted_pings,
            count(distinct case when event_type = 'order_already_accepted' then unique_ping_level_id end) as already_accepted_pings,
            count(distinct case when event_type = 'rider_accept_failed' then unique_ping_level_id end) as rider_accept_failure_pings,        
            count(distinct case when event_type = 'rider_acknowledged' then unique_ping_level_id end) as rider_ack_pings
            
            from
            (
              select t1.yyyymmdd,test_ctrl,t1.time_period,data_riderId,data_orderid,
                t1.pickup_location_hex_8,event_type,unique_ping_level_id,hex_data.city_name

                from
                (
                    select propagation_batch_id as data_propagationBatchId,rider_id as data_riderId,order_id as data_orderid,yyyymmdd,quarter_hour,hh,
                    concat(propagation_batch_id,rider_id,order_id) as unique_ping_level_id,pickup_location_hex_8,
                    updated_epoch as epoch,event_type,
                    CASE 
                        WHEN substr(quarter_hour,1,2) IN ('08','09','10','11') THEN 'morning_peak'
                        WHEN substr(quarter_hour,1,2) IN ('12','13','14','15','16') THEN 'afternoon'
                        WHEN substr(quarter_hour,1,2) IN ('17','18','19','20','21') THEN 'evening_peak'
                        WHEN substr(quarter_hour,1,2) IN ('00','01','02','03','04','05','06','07') THEN 'rest_morning'        
                        ELSE 'rest_evening' END AS time_period
                        
                    from orders.dispatch_propagation_immutable
                    --where yyyymmdd = cast({dt} as varchar)
                    where yyyymmdd >= '20230704' and yyyymmdd <= '20230722'
                    and order_id in (select distinct order_id from order_filtering)
                    and event_type in ('rider_acknowledged')
                    and propagation_type in ('multicast')

                    union all

                    select data_propagationBatchId,data_riderId,data_orderid,yyyymmdd,quarter_hour,hh,
                    concat(data_propagationBatchId,data_riderId,data_orderid) as unique_ping_level_id,pickup_location_hex_8,
                    updated_epoch as epoch,event_type,time_period


                    from(
                            select event_type as eventtype,propagation_batch_id as data_propagationBatchId,rider_id as data_riderId,order_id as data_orderid,yyyymmdd,quarter_hour,hh,
                            updated_epoch,event_type,pickup_location_hex_8,drop_location_hex_8,epoch,
                            row_number() over(partition by propagation_batch_id,rider_id,order_id order by epoch desc) as row,
                            CASE 
                                WHEN substr(quarter_hour,1,2) IN ('08','09','10','11') THEN 'morning_peak'
                                WHEN substr(quarter_hour,1,2) IN ('12','13','14','15','16') THEN 'afternoon'
                                WHEN substr(quarter_hour,1,2) IN ('17','18','19','20','21') THEN 'evening_peak'
                                WHEN substr(quarter_hour,1,2) IN ('00','01','02','03','04','05','06','07') THEN 'rest_morning'        
                                ELSE 'rest_evening' END AS time_period
            
                            from orders.dispatch_propagation_immutable
                                --where yyyymmdd = cast({dt} as varchar)
                                where yyyymmdd >= '20230704' and yyyymmdd <= '20230722'
                                and order_id in (select distinct order_id from order_filtering)
                                and propagation_type in ('multicast')
                                and event_type in ('rider_busy','order_already_accepted','rider_accept_failed','rider_rejected','rider_accepted','rider_ineligible','rider_unreachable')
                        ) base

                        where row =1
                        
                    ) as t1
                

                join (select city_name,test_ctrl, hex_id ,time_bucket
                            from experiments_internal.auto_apr_fm_reduction_exp_230704 
                            where city_name in ('Pune','Hyderabad') ) hex_data

                on t1.pickup_location_hex_8=hex_data.hex_id and t1.time_period = hex_data.time_bucket 

               
               where case when t1.yyyymmdd >='20230710' and concat(t1.pickup_location_hex_8,t1.time_period) 
               in ('8860a25829fffffevening_peak','8860a259c1fffffevening_peak',
                    '8860a2583bfffffevening_peak','8860a2598dfffffevening_peak',
                    '8860a259cdfffffevening_peak','8860a25825fffffevening_peak',
                    '8860a259d5fffffevening_peak','8860a25839fffffevening_peak',
                    '8860a25821fffffevening_peak','8860a25989fffffevening_peak',
                    '8860a2583dfffffevening_peak','8860a259e1fffffevening_peak',
                    '8860a259ddfffffevening_peak','8860a25833fffffevening_peak',
                    '8860a25827fffffevening_peak','8860a259c7fffffevening_peak',
                    '8860a259c9fffffevening_peak','8860a25835fffffevening_peak',
                    '8860a25917fffffevening_peak','8860a259d1fffffevening_peak',
                    '8860a2591bfffffevening_peak','8860a259e3fffffevening_peak',
                    '8860a25831fffffevening_peak','8860a25913fffffevening_peak',
                    '8860a25807fffffevening_peak','8860a259e9fffffevening_peak',
                    '8860a2582bfffffevening_peak','8860a25823fffffevening_peak',
                    '8860a25837fffffevening_peak','8860a259edfffffevening_peak',
                    '8860a259c3fffffevening_peak','8860a25911fffffevening_peak',
                    '8860a259cbfffffevening_peak','8860a259ebfffffevening_peak',
                    '8860a25919fffffevening_peak','8860a259d9fffffevening_peak',
                    '8860a259c5fffffevening_peak','8860b19693fffffevening_peak','8860a24b45fffffevening_peak','8860b194d9fffffevening_peak',
                    '8860a24b47fffffevening_peak','8860a24b01fffffevening_peak','8860b19695fffffevening_peak','8860b19697fffffevening_peak',
                    '8860a24b21fffffevening_peak','8860a24b2bfffffevening_peak','8860a24b63fffffevening_peak','8860b194dbfffffevening_peak',
                    '8860a24b29fffffevening_peak','8860a24b09fffffevening_peak','8860a24b69fffffevening_peak','8860a24b23fffffevening_peak',
                    '8860a24b05fffffevening_peak','8860b196bbfffffevening_peak','8860a24b2dfffffevening_peak','8860a24b65fffffevening_peak',
                    '8860b19691fffffevening_peak','8860a24b67fffffevening_peak','8860a24b6dfffffevening_peak','8860b196b3fffffevening_peak',
                    '8860b196b7fffffevening_peak','8860a24b0dfffffevening_peak','8860b196b1fffffevening_peak','8860a24b6bfffffevening_peak',
                    '8860a24b61fffffevening_peak','8860a24a23fffffmorning_peak','8860a24a05fffffmorning_peak','8860a24b13fffffmorning_peak',
                    '8860a24845fffffmorning_peak','8860a24a2bfffffmorning_peak','8860a24a0dfffffmorning_peak','8860a24b55fffffmorning_peak',
                    '8860a24b11fffffmorning_peak','8860a24a6dfffffmorning_peak','8860a24b0bfffffmorning_peak','8860a259b3fffffmorning_peak',
                    '8860a24b03fffffmorning_peak','8860a259b7fffffmorning_peak','8860a24841fffffmorning_peak','8860a24a27fffffmorning_peak',
                    '8860a24a21fffffmorning_peak','8860a24849fffffmorning_peak','8860a24a63fffffmorning_peak','8860a24b15fffffmorning_peak',
                    '8860a24a69fffffmorning_peak','8860a24b1dfffffmorning_peak','8860a24b51fffffmorning_peak','8860a24a61fffffmorning_peak',
                    '8860a24a67fffffmorning_peak','8860a25993fffffmorning_peak','8860a24869fffffmorning_peak','8860a24b59fffffmorning_peak',
                    '8860a24a2dfffffmorning_peak','8860a24a6bfffffmorning_peak','8860a24a25fffffmorning_peak','8860a24b1bfffffmorning_peak',
                    '8860a24b19fffffmorning_peak','8860a2484dfffffmorning_peak','8860a24a29fffffmorning_peak','8860a24b57fffffmorning_peak',
                    '8860a24b5dfffffmorning_peak','8860a24b53fffffmorning_peak','8860a24b17fffffmorning_peak','8860a24b5bfffffmorning_peak',
                    '8860a24a65fffffmorning_peak'
                    
                    ) then 1 else 0 end =0
                
                and 
                   concat(t1.time_period,t1.pickup_location_hex_8) not in 
                    ('evening_peak8860b19693fffff','evening_peak8860a24b45fffff','evening_peak8860b194d9fffff',
                    'evening_peak8860a24b47fffff','evening_peak8860a24b01fffff','evening_peak8860b19695fffff',
                    'evening_peak8860b19697fffff','evening_peak8860a24b21fffff','evening_peak8860a24b2bfffff',
                    'evening_peak8860a24b63fffff','evening_peak8860b194dbfffff','evening_peak8860a24b29fffff',
                    'evening_peak8860a24b09fffff','evening_peak8860a24b69fffff','evening_peak8860a24b23fffff',
                    'evening_peak8860a24b05fffff','evening_peak8860b196bbfffff','evening_peak8860a24b2dfffff',
                    'evening_peak8860a24b65fffff','evening_peak8860b19691fffff','evening_peak8860a24b67fffff',
                    'evening_peak8860a24b6dfffff','evening_peak8860b196b3fffff','evening_peak8860b196b7fffff',
                    'evening_peak8860a24b0dfffff','evening_peak8860b196b1fffff','evening_peak8860a24b6bfffff','evening_peak8860a24b61fffff'
                    )
               
            )
            
            group by 1,2,3,4,5
            
        ),
        
 order_one as 
             (
            select city_name,test_ctrl,captain_id,sum(net_orders) as net_orders--,avg(net_orders) as rpr
            from
            (
            select yyyymmdd,v1.city_name,test_ctrl,captain_id,pickup_location_hex_8,time_period,net_orders
            from
                   (select yyyymmdd,city_name,captain_id,pickup_location_hex_8,
                    CASE 
                    WHEN substr(quarter_hour,1,2) IN ('08','09','10','11') THEN 'morning_peak'
                    WHEN substr(quarter_hour,1,2) IN ('12','13','14','15','16') THEN 'afternoon'
                    WHEN substr(quarter_hour,1,2) IN ('17','18','19','20','21') THEN 'evening_peak'
                    WHEN substr(quarter_hour,1,2) IN ('00','01','02','03','04','05','06','07') THEN 'rest_morning'        
                    ELSE 'rest_evening' END AS time_period,
                    count(distinct order_id) as net_orders
                   
                    from  orders.order_logs_snapshot
                    where yyyymmdd >= '20230704' and yyyymmdd <= '20230722'
                    and order_status='dropped'
                    and service_obj_service_name = 'Auto'
                    and (spd_fraud_flag = false OR spd_fraud_flag IS NULL)
                    and city_name in ('Hyderabad','Pune')
                    
                    group by 1,2,3,4,5 
                    ) as v1
               
                join (select city_name,test_ctrl, hex_id ,time_bucket
                            from experiments_internal.auto_apr_fm_reduction_exp_230704 
                            where city_name in ('Pune','Hyderabad') ) hex_data

                on v1.pickup_location_hex_8=hex_id and v1.time_period = time_bucket 
            )
            group by 1,2,3       
    
               
        )
            
        select city_name,data_riderId,
        sum(case when test_ctrl in ('ctrl_1','ctrl_4','ctrl_3') then accepted_pings end) as ctrl_accp,
        sum(case when test_ctrl='test_2' then accepted_pings end) as test_accp,

        sum(case when test_ctrl in ('ctrl_1','ctrl_4','ctrl_3') then total_pings end) as ctrl_ttl_pings,
        sum(case when test_ctrl='test_2' then total_pings end) as test_ttl_pings,
        
        sum(total_pings) as total_pings
        
        --sum(case when test_ctrl in ('ctrl_1','ctrl_4','ctrl_3') then net_orders end) as ctrl_net_orders,
        --sum(case when test_ctrl='test_2' then total_pings end) as test_net_orders--,
        
        --avg(case when test_ctrl in ('ctrl_1','ctrl_4','ctrl_3') then net_orders end) as ctrl_rpr,
        --avg(case when test_ctrl='test_2' then total_pings end) as test_rpr
        
       from
        (
           select 
           
               t1.city_name,t1.test_ctrl,data_riderId,--net_orders,--rpr,
               sum(accepted_pings+rider_accept_failure_pings) as accepted_pings,
               sum(rider_ack_pings) as total_pings
               
               from t1
               
               --left join order_one
               --on t1.test_ctrl=order_one.test_ctrl and t1.data_riderId = order_one.captain_id
  
               group by 1,2,3
       )
       group by 1,2


"""
df_test = pd.read_sql_query(query, metabase_connection)

#                   --case when yyyymmdd<='20230704' then 'pre' else 'post' end as pre_post,

In [54]:
df_test

,city_name,data_riderId,ctrl_accp,test_accp,ctrl_ttl_pings,test_ttl_pings,total_pings
0,Hyderabad,5f644c6c6718a937fafb6d70,108.0,52.0,3280.0,991.0,4271
1,Hyderabad,5f3ccb5cce14713ccd7c5f98,41.0,13.0,1333.0,297.0,1630
2,Hyderabad,5f6ae0239025c2327e40bf07,38.0,9.0,528.0,212.0,740
3,Hyderabad,5d51756c55fbf50d45f12b49,10.0,1.0,137.0,28.0,165
4,Hyderabad,5c46dac74a267149c77768b3,17.0,2.0,238.0,65.0,303
...,...,...,...,...,...,...,...
84845,Hyderabad,5f17e310ffa9145543c8c88f,9.0,1.0,290.0,105.0,395
84846,Hyderabad,6200cce03073731f91f295cb,3.0,1.0,30.0,4.0,34
84847,Hyderabad,5cdfd12d25ee3218d4cdb816,0.0,0.0,2.0,1.0,3
84848,Hyderabad,63857f80a56eb36d38ce0007,0.0,NaN,26.0,NaN,26


In [55]:
#post
df_test=df_test.rename(columns={'ctrl_ttl_pings':'post_ctrl_ttl_pings','test_ttl_pings':'post_test_ttl_pings',
                               })

In [56]:
df_test['post_ctrl_pings_share']=df_test.post_ctrl_ttl_pings/df_test.total_pings
df_test['post_test_pings_share']=df_test.post_test_ttl_pings/df_test.total_pings

new_2=df_test[['city_name','data_riderId','post_ctrl_pings_share','post_test_pings_share','post_ctrl_ttl_pings','post_test_ttl_pings']]#.post_ctrl_pings_share



In [57]:
new_2.head()

,city_name,data_riderId,post_ctrl_pings_share,post_test_pings_share,post_ctrl_ttl_pings,post_test_ttl_pings
0,Hyderabad,5f644c6c6718a937fafb6d70,0.767970,0.232030,3280.0,991.0
1,Hyderabad,5f3ccb5cce14713ccd7c5f98,0.817791,0.182209,1333.0,297.0
2,Hyderabad,5f6ae0239025c2327e40bf07,0.713514,0.286486,528.0,212.0
3,Hyderabad,5d51756c55fbf50d45f12b49,0.830303,0.169697,137.0,28.0
4,Hyderabad,5c46dac74a267149c77768b3,0.785479,0.214521,238.0,65.0


In [ ]:
#pre

In [46]:
df_test=df_test.rename(columns={'ctrl_ttl_pings':'pre_ctrl_ttl_pings','test_ttl_pings':'pre_test_ttl_pings',
                               })

In [47]:
df_test['pre_ctrl_APR']=round(df_test.ctrl_accp/df_test.pre_ctrl_ttl_pings,2)
df_test['pre_test_APR']=round(df_test.test_accp/df_test.pre_test_ttl_pings,2)


In [48]:

pre_data=df_test[['city_name', 'data_riderId','pre_ctrl_APR','pre_test_APR','pre_ctrl_ttl_pings','pre_test_ttl_pings']]



In [58]:
import copy
pre_data_one=copy.deepcopy(pre_data)

In [59]:
pre_data_one['pre_post']='pre'

In [60]:
pre_data_one.head()

,city_name,data_riderId,pre_ctrl_APR,pre_test_APR,pre_ctrl_ttl_pings,pre_test_ttl_pings,pre_post
0,Hyderabad,6356957ece6cf87dfbb78839,0.02,0.02,6990.0,1665.0,pre
1,Hyderabad,5f3cee63979c25bb70019d1e,0.04,0.06,800.0,285.0,pre
2,Pune,63ce1560f8873d628e203ea7,0.19,0.35,52.0,20.0,pre
3,Pune,6184c92a57f1441e12b4c882,0.05,0.05,750.0,200.0,pre
4,Hyderabad,5e72ed6c6b303470d553dbb7,0.02,0.02,985.0,442.0,pre


In [61]:
#merge pre and post

In [62]:
fnl_dt=pre_data_one.merge(new_2,on=['city_name','data_riderId'],how='left')

In [63]:
fnl_dt=fnl_dt.drop(columns=['pre_post'])

In [64]:
fnl_dt.columns#.isnull()

Index(['city_name', 'data_riderId', 'pre_ctrl_APR', 'pre_test_APR',
       'pre_ctrl_ttl_pings', 'pre_test_ttl_pings', 'post_ctrl_pings_share',
       'post_test_pings_share', 'post_ctrl_ttl_pings', 'post_test_ttl_pings'],
      dtype='object')

In [65]:
fnl_dt

,city_name,data_riderId,pre_ctrl_APR,pre_test_APR,pre_ctrl_ttl_pings,pre_test_ttl_pings,post_ctrl_pings_share,post_test_pings_share,post_ctrl_ttl_pings,post_test_ttl_pings
0,Hyderabad,6356957ece6cf87dfbb78839,0.02,0.02,6990.0,1665.0,0.888004,0.111996,4908.0,619.0
1,Hyderabad,5f3cee63979c25bb70019d1e,0.04,0.06,800.0,285.0,0.805310,0.194690,455.0,110.0
2,Pune,63ce1560f8873d628e203ea7,0.19,0.35,52.0,20.0,0.641509,0.358491,34.0,19.0
3,Pune,6184c92a57f1441e12b4c882,0.05,0.05,750.0,200.0,0.850962,0.149038,1062.0,186.0
4,Hyderabad,5e72ed6c6b303470d553dbb7,0.02,0.02,985.0,442.0,0.821571,0.178429,1851.0,402.0
...,...,...,...,...,...,...,...,...,...,...
79071,Pune,61bd714efaf602517f3c3df4,0.00,NaN,14.0,0.0,NaN,NaN,NaN,NaN
79072,Pune,62a9efc517af9a2123587c36,0.00,0.00,4.0,12.0,0.518519,0.481481,14.0,13.0
79073,Hyderabad,5b87f0d2581a362e8c22fe0d,0.00,NaN,1.0,NaN,NaN,NaN,NaN,NaN
79074,Pune,6491985490c05e0e653dd603,NaN,0.00,NaN,1.0,NaN,NaN,NaN,NaN


In [68]:
fnl_dt2=fnl_dt[['pre_ctrl_APR', 'pre_test_APR',
       'pre_ctrl_ttl_pings', 'pre_test_ttl_pings', 'post_ctrl_pings_share',
       'post_test_pings_share', 'post_ctrl_ttl_pings', 'post_test_ttl_pings']].dropna()

In [69]:
fnl_dt3=fnl_dt[['city_name','data_riderId']].merge(fnl_dt2,left_index=True,right_index=True)

In [70]:
fnl_dt3

,city_name,data_riderId,pre_ctrl_APR,pre_test_APR,pre_ctrl_ttl_pings,pre_test_ttl_pings,post_ctrl_pings_share,post_test_pings_share,post_ctrl_ttl_pings,post_test_ttl_pings
0,Hyderabad,6356957ece6cf87dfbb78839,0.02,0.02,6990.0,1665.0,0.888004,0.111996,4908.0,619.0
1,Hyderabad,5f3cee63979c25bb70019d1e,0.04,0.06,800.0,285.0,0.805310,0.194690,455.0,110.0
2,Pune,63ce1560f8873d628e203ea7,0.19,0.35,52.0,20.0,0.641509,0.358491,34.0,19.0
3,Pune,6184c92a57f1441e12b4c882,0.05,0.05,750.0,200.0,0.850962,0.149038,1062.0,186.0
4,Hyderabad,5e72ed6c6b303470d553dbb7,0.02,0.02,985.0,442.0,0.821571,0.178429,1851.0,402.0
...,...,...,...,...,...,...,...,...,...,...
79064,Hyderabad,620133a4d2ae404bd3983d2e,0.05,0.04,1433.0,590.0,0.845103,0.154897,2968.0,544.0
79066,Hyderabad,60e2e56109aa990a454048c8,0.00,0.00,5.0,2.0,0.888889,0.111111,8.0,1.0
79067,Hyderabad,639860c7f980205084f320d6,0.02,0.02,704.0,312.0,0.751064,0.248936,706.0,234.0
79068,Pune,61a5dac217dd6aa6c3059598,0.03,0.02,130.0,56.0,0.734884,0.265116,158.0,57.0


In [ ]:
#  pre
#   - apr
#   - total_pings
#  post 
#   - exposure - share of pings, total_pings


In [71]:
hyd_data=fnl_dt3[fnl_dt3.city_name=='Hyderabad']

In [73]:
hyd_data[hyd_data.city_name=='Hyderabad'].describe()

,pre_ctrl_APR,pre_test_APR,pre_ctrl_ttl_pings,pre_test_ttl_pings,post_ctrl_pings_share,post_test_pings_share,post_ctrl_ttl_pings,post_test_ttl_pings
count,43890.000000,43890.000000,43890.000000,43890.000000,43890.000000,43890.000000,43890.000000,43890.000000
mean,0.057710,0.059156,832.301504,287.709273,0.813264,0.186736,1379.574755,297.023331
std,0.056364,0.067103,849.115495,304.295404,0.068642,0.068642,1364.473458,297.318832
min,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,0.020000,0.020000,208.000000,69.000000,0.787997,0.149117,361.000000,79.000000
50%,0.040000,0.040000,598.000000,202.000000,0.821588,0.178412,1011.000000,219.000000
75%,0.080000,0.080000,1185.000000,406.000000,0.850883,0.212003,1966.000000,423.000000
max,1.000000,1.000000,10796.000000,4641.000000,1.000000,1.000000,15239.000000,4613.000000


In [ ]:
#high threshold for high test exposure  (50% share), 

In [327]:
hyd_data['pre_ctrl_APR_buckets']=hyd_data.pre_ctrl_APR.apply(lambda x: 'pre_ctrl_apr_0_4_per' if x<= 0.04 else ('pre_ctrl_apr_4_8_per' if x<=0.08 else 
                                                                      'pre_ctrl_apr_8_above_per') )

In [328]:
hyd_data['pre_test_APR_buckets']=hyd_data.pre_test_APR.apply(lambda x: 'pre_test_apr_0_4_per' if x<= 0.04 else ('pre_test_apr_4_8_per' if x<=0.08 else 
                                                                      'pre_test_apr_8_above_per') )

In [329]:
hyd_data['pre_ctrl_ttl_pings_buckets']=hyd_data.pre_ctrl_ttl_pings.apply(lambda x: 'pre_cntrl_ttl_pings_bucket_one' 
                                 if x<= 612 else ('pre_cntrl_ttl_pings_bucket_two' 
                                                 if x<=1199 else 
                                                'pre_cntrl_ttl_pings_bucket_three') )

In [330]:
hyd_data['pre_test_ttl_pings_buckets']=hyd_data.pre_test_ttl_pings.apply(lambda x: 'pre_test_ttl_pings_bucket_one' 
                                 if x<= 206 else ('pre_test_ttl_pings_bucket_two' 
                                                 if x<=410 else 
                                                'pre_test_ttl_pings_bucket_three') )

In [335]:
hyd_data['post_ctrl_ttl_pings_buckets']=hyd_data.post_ctrl_ttl_pings.apply(lambda x: 'post_cntrl_ttl_pings_bucket_one' 
                                 if x<= 787 else ('post_cntrl_ttl_pings_bucket_two' 
                                                 if x<=1529 else 
                                                'post_cntrl_ttl_pings_bucket_three') )

In [336]:
hyd_data['post_test_ttl_pings_buckets']=hyd_data.post_test_ttl_pings.apply(lambda x: 'post_test_ttl_pings_bucket_one' 
                                 if x<= 173 else ('post_test_ttl_pings_bucket_two' 
                                                 if x<=337 else 
                                                'post_test_ttl_pings_bucket_three') )

In [337]:
hyd_data['post_ctrl_pings_share_buckets']=hyd_data.post_ctrl_pings_share.apply(lambda x: 'post_cntrl_pings_share_bucket_one' 
                                 if x<= 0.81 else ('post_cntrl_pings_share_bucket_two' 
                                                 if x<=0.85 else 
                                                'post_cntrl_pings_share_bucket_three') )



In [338]:
hyd_data['post_test_pings_share_buckets']=hyd_data.post_test_pings_share.apply(lambda x: 'post_test_pings_share_bucket_one' 
                                 if x<= 0.18 else ('post_test_pings_share_bucket_two' 
                                                 if x<=0.21 else 
                                                'post_test_pings_share_bucket_three') )



In [339]:
pune_data=fnl_dt3[fnl_dt3.city_name=='Pune']

In [341]:
pune_data[pune_data.city_name=='Pune'].describe()

,pre_ctrl_APR,pre_test_APR,pre_ctrl_ttl_pings,pre_test_ttl_pings,post_ctrl_pings_share,post_test_pings_share,post_ctrl_ttl_pings,post_test_ttl_pings
count,24238.000000,24238.000000,24238.000000,24238.000000,24238.000000,24238.000000,24238.000000,24238.000000
mean,0.063851,0.067216,329.184050,95.242471,0.785425,0.214575,347.884603,87.919176
std,0.073971,0.092022,321.847282,91.197121,0.092817,0.092817,341.696940,85.104184
min,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,0.020000,0.010000,90.000000,26.000000,0.746421,0.159091,98.000000,24.000000
50%,0.050000,0.040000,239.000000,70.000000,0.796844,0.203156,253.000000,65.000000
75%,0.080000,0.090000,467.000000,137.000000,0.840909,0.253579,492.000000,125.000000
max,1.000000,1.000000,3041.000000,950.000000,1.000000,1.000000,4369.000000,1165.000000


In [342]:
pune_data['pre_ctrl_APR_buckets']=pune_data.pre_ctrl_APR.apply(lambda x: 'pre_ctrl_apr_0_5_per' if x<= 0.05 else ('pre_ctrl_apr_5_8_per' if x<=0.08 else 
                                                                      'pre_ctrl_apr_8_above_per') )

pune_data['pre_test_APR_buckets']=pune_data.pre_test_APR.apply(lambda x: 'pre_test_apr_0_4_per' if x<= 0.04 else ('pre_test_apr_4_9_per' if x<=0.09 else 
                                                                      'pre_test_apr_8_above_per') )

In [343]:
pune_data['pre_ctrl_ttl_pings_buckets']=pune_data.pre_ctrl_ttl_pings.apply(lambda x: 'pre_cntrl_ttl_pings_bucket_one' 
                                 if x<= 239 else ('pre_cntrl_ttl_pings_bucket_two' 
                                                 if x<=467 else 
                                                'pre_cntrl_ttl_pings_bucket_three') )


pune_data['pre_test_ttl_pings_buckets']=pune_data.pre_test_ttl_pings.apply(lambda x: 'pre_test_ttl_pings_bucket_one' 
                                 if x<= 70 else ('pre_test_ttl_pings_bucket_two' 
                                                 if x<=137 else 
                                                'pre_test_ttl_pings_bucket_three') )

In [344]:
pune_data['post_ctrl_ttl_pings_buckets']=pune_data.post_ctrl_ttl_pings.apply(lambda x: 'post_cntrl_ttl_pings_bucket_one' 
                                 if x<= 253 else ('post_cntrl_ttl_pings_bucket_two' 
                                                 if x<=492 else 
                                                'post_cntrl_ttl_pings_bucket_three') )


pune_data['post_test_ttl_pings_buckets']=pune_data.post_test_ttl_pings.apply(lambda x: 'post_test_ttl_pings_bucket_one' 
                                 if x<= 65 else ('post_test_ttl_pings_bucket_two' 
                                                 if x<=125 else 
                                                'post_test_ttl_pings_bucket_three') )


In [345]:
pune_data['post_ctrl_pings_share_buckets']=pune_data.post_ctrl_pings_share.apply(lambda x: 'post_cntrl_pings_share_bucket_one' 
                                 if x<= 0.79 else ('post_cntrl_pings_share_bucket_two' 
                                                 if x<=0.84 else 
                                                'post_cntrl_pings_share_bucket_three') )


pune_data['post_test_pings_share_buckets']=pune_data.post_test_pings_share.apply(lambda x: 'post_test_pings_share_bucket_one' 
                                 if x<= 0.20 else ('post_test_pings_share_bucket_two' 
                                                 if x<=0.25 else 
                                                'post_test_pings_share_bucket_three') )




### hyd getting exposure

In [398]:
hyd_data[[
#     'pre_ctrl_ttl_pings_buckets','pre_ctrl_APR_buckets', 
    'post_ctrl_ttl_pings_buckets','post_ctrl_pings_share_buckets']].value_counts().reset_index().iloc[:20]



,post_ctrl_ttl_pings_buckets,post_ctrl_pings_share_buckets,0
0,post_cntrl_ttl_pings_bucket_one,post_cntrl_pings_share_bucket_one,10396
1,post_cntrl_ttl_pings_bucket_one,post_cntrl_pings_share_bucket_three,5996
2,post_cntrl_ttl_pings_bucket_one,post_cntrl_pings_share_bucket_two,5140
3,post_cntrl_ttl_pings_bucket_three,post_cntrl_pings_share_bucket_two,4826
4,post_cntrl_ttl_pings_bucket_two,post_cntrl_pings_share_bucket_one,4292
5,post_cntrl_ttl_pings_bucket_two,post_cntrl_pings_share_bucket_two,4169
6,post_cntrl_ttl_pings_bucket_three,post_cntrl_pings_share_bucket_one,3406
7,post_cntrl_ttl_pings_bucket_three,post_cntrl_pings_share_bucket_three,2526
8,post_cntrl_ttl_pings_bucket_two,post_cntrl_pings_share_bucket_three,2304


In [ ]:
# post_cntrl_ttl_pings_bucket_three,post_cntrl_pings_share_bucket_three - 2526 high exposure to control (5.8%)
# post_cntrl_ttl_pings_bucket_three,post_cntrl_pings_share_bucket_one - 3406 high exposure to test (7.9%)

In [401]:
xx2526/len(hyd_data)

0.05866914411798862

In [402]:
3406/len(hyd_data)

0.07910811752409709

In [408]:
for_hyd_exp1=hyd_data[(hyd_data.post_ctrl_ttl_pings_buckets=='post_cntrl_ttl_pings_bucket_three')&
        (hyd_data.post_ctrl_pings_share_buckets=='post_cntrl_pings_share_bucket_three')].reset_index(drop=True)

for_hyd_exp1['exposure']='low_test_exposure'
for_hyd_exp1_sub=for_hyd_exp1[['city_name','data_riderId','exposure']]

In [414]:
for_hyd_exp2=hyd_data[(hyd_data.post_ctrl_ttl_pings_buckets=='post_cntrl_ttl_pings_bucket_three')&
        (hyd_data.post_ctrl_pings_share_buckets=='post_cntrl_pings_share_bucket_one')].reset_index(drop=True)

for_hyd_exp2['exposure']='high_test_exposure'
for_hyd_exp2_sub=for_hyd_exp2[['city_name','data_riderId','exposure']]

In [417]:
hyd_true_fnl=pd.concat([for_hyd_exp1_sub,for_hyd_exp2_sub]).reset_index(drop=True)

In [418]:
hyd_true_fnl.head()

,city_name,data_riderId,exposure
0,Hyderabad,6299e07f22577d53bafab537,low_test_exposure
1,Hyderabad,62d98e260037b14d09d01a33,low_test_exposure
2,Hyderabad,5f82aa12400204f25057a8bd,low_test_exposure
3,Hyderabad,649679b24e8ea9b1c93a624e,low_test_exposure
4,Hyderabad,647c378705281a236c020fc1,low_test_exposure


In [419]:
pune_data[[
#     'pre_ctrl_ttl_pings_buckets','pre_ctrl_APR_buckets', 
    'post_ctrl_ttl_pings_buckets','post_ctrl_pings_share_buckets']].value_counts().reset_index().iloc[:20]


# post_cntrl_ttl_pings_bucket_three,post_cntrl_pings_share_bucket_three - 1636 high exposure to control (6.7%)
# post_cntrl_ttl_pings_bucket_three,post_cntrl_pings_share_bucket_one - 2107 high exposure to test (8.6%)

,post_ctrl_ttl_pings_buckets,post_ctrl_pings_share_buckets,0
0,post_cntrl_ttl_pings_bucket_one,post_cntrl_pings_share_bucket_one,6337
1,post_cntrl_ttl_pings_bucket_one,post_cntrl_pings_share_bucket_three,3212
2,post_cntrl_ttl_pings_bucket_two,post_cntrl_pings_share_bucket_one,2731
3,post_cntrl_ttl_pings_bucket_one,post_cntrl_pings_share_bucket_two,2581
4,post_cntrl_ttl_pings_bucket_three,post_cntrl_pings_share_bucket_two,2307
5,post_cntrl_ttl_pings_bucket_three,post_cntrl_pings_share_bucket_one,2107
6,post_cntrl_ttl_pings_bucket_two,post_cntrl_pings_share_bucket_two,2021
7,post_cntrl_ttl_pings_bucket_three,post_cntrl_pings_share_bucket_three,1636
8,post_cntrl_ttl_pings_bucket_two,post_cntrl_pings_share_bucket_three,1306


In [420]:
for_pune_exp1=pune_data[(pune_data.post_ctrl_ttl_pings_buckets=='post_cntrl_ttl_pings_bucket_three')&
        (pune_data.post_ctrl_pings_share_buckets=='post_cntrl_pings_share_bucket_three')].reset_index(drop=True)

for_pune_exp1['exposure']='low_test_exposure'
for_pune_exp1_sub=for_pune_exp1[['city_name','data_riderId','exposure']]

In [421]:
for_pune_exp2=pune_data[(pune_data.post_ctrl_ttl_pings_buckets=='post_cntrl_ttl_pings_bucket_three')&
        (pune_data.post_ctrl_pings_share_buckets=='post_cntrl_pings_share_bucket_one')].reset_index(drop=True)

for_pune_exp2['exposure']='high_test_exposure'
for_pune_exp2_sub=for_pune_exp2[['city_name','data_riderId','exposure']]

In [422]:
pune_true_fnl=pd.concat([for_pune_exp1_sub,for_pune_exp2_sub]).reset_index(drop=True)

In [426]:
pune_true_fnl.head()

,city_name,data_riderId,exposure
0,Pune,63d8b295ffad66cd75c22fd8,low_test_exposure
1,Pune,6194a781ae4f738e9a1c1fb1,low_test_exposure
2,Pune,6194b017c98cb3715d024125,low_test_exposure
3,Pune,6191f0b1c98cb3ab3c00028e,low_test_exposure
4,Pune,64004b1fa66e7444da11c84f,low_test_exposure


In [428]:
total_exp_captains=pd.concat([hyd_true_fnl,pune_true_fnl]).reset_index(drop=True)

In [431]:
total_exp_captains=total_exp_captains.rename(columns={'data_riderId':'captain_id'})

In [432]:
total_exp_captains.head()

,city_name,captain_id,exposure
0,Hyderabad,6299e07f22577d53bafab537,low_test_exposure
1,Hyderabad,62d98e260037b14d09d01a33,low_test_exposure
2,Hyderabad,5f82aa12400204f25057a8bd,low_test_exposure
3,Hyderabad,649679b24e8ea9b1c93a624e,low_test_exposure
4,Hyderabad,647c378705281a236c020fc1,low_test_exposure


In [433]:
total_exp_captains.to_csv('fm_exposure_captain_list.csv',index=False)

In [140]:
df_test['exposure']=df_test.test_pings_by_captain.apply(lambda x: 'high_exposure' if x>=98 else 'low_exposure')

In [141]:
df_test=df_test.rename(columns={'exposure':'test_hexs_exposure','data_riderId':'captain_id'}).reset_index(drop=True)

In [142]:
df_test[['captain_id','test_hexs_exposure']].to_csv('fm_limit_exposure_pune.csv',index=False)

In [160]:
df_test[df_test.test_hexs_exposure=='high_exposure']

,city_name,yyyymmdd,test_ctrl,time_period,test_hexs_exposure,marketplace_segment,pc_segment,LM_bucket,rider_busy_pings,rider_rejeted_pings,accepted_pings,all_accepted_pings,already_accepted_pings,total_pings
1,Pune,20230707,ctrl_3,evening_peak,high_exposure,balanced-low,HP_Daily,2-5,64.0,110.0,7.0,7.0,17.0,209.0
5,Hyderabad,20230707,ctrl_3,afternoon,high_exposure,balanced-low,LP_intra,<=2,61.0,88.0,3.0,3.0,78.0,232.0
10,Pune,20230707,ctrl_1,morning_peak,high_exposure,out-low,MP_intra,8-12,20.0,5.0,3.0,7.0,27.0,60.0
12,Hyderabad,20230707,ctrl_1,evening_peak,high_exposure,out-high,UHP_Daily,8-12,84.0,294.0,5.0,6.0,19.0,428.0
15,Hyderabad,20230707,test_2,evening_peak,high_exposure,balanced-high,UHP_Daily,<=2,32.0,67.0,9.0,9.0,4.0,118.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23524,Pune,20230707,ctrl_1,evening_peak,high_exposure,out-high,LP_inter,>12,1.0,0.0,1.0,1.0,2.0,4.0
23525,Pune,20230707,None,rest_evening,high_exposure,in-low,MP_inter,8-12,0.0,0.0,0.0,0.0,3.0,3.0
23527,Pune,20230707,ctrl_3,rest_evening,high_exposure,in-low,LP_Daily,None,0.0,1.0,0.0,0.0,0.0,1.0
23529,Pune,20230707,ctrl_4,afternoon,high_exposure,balanced-low,UHP_intra,>12,0.0,0.0,0.0,0.0,1.0,1.0


In [146]:
dt1=pd.DataFrame()

for i in pd.date_range('20230704','20230707'):
    

    query="""
    with order_filtering as 
    (
        select *,      case 
                        when rounded_last_mile <= 2 then '<=2'
                        when rounded_last_mile >2 and rounded_last_mile <=5 then '2-5'
                        when rounded_last_mile >5 and rounded_last_mile <=8 then '5-8'
                        when rounded_last_mile >8 and rounded_last_mile <=12 then '8-12'
                        when rounded_last_mile > 12 then '>12' end as LM_bucket
        from
        (
        select distinct yyyymmdd,order_id ,city_name, pickup_location_hex_8,
        case when distance_final_distance<0.5 then round(distance_final_distance)+1 
        else round(distance_final_distance) end as rounded_last_mile,
        CASE 
            WHEN substr(quarter_hour,1,2) IN ('08','09','10','11') THEN 'morning_peak'
            WHEN substr(quarter_hour,1,2) IN ('12','13','14','15','16') THEN 'afternoon'
            WHEN substr(quarter_hour,1,2) IN ('17','18','19','20','21') THEN 'evening_peak'
            WHEN substr(quarter_hour,1,2) IN ('00','01','02','03','04','05','06','07') THEN 'rest_morning'        
            ELSE 'rest_evening' END AS time_period, order_status,quarter_hour

        from 
            orders.order_logs_snapshot
            where yyyymmdd = cast({dt} as varchar)
            --where yyyymmdd >= '20230620' and yyyymmdd <= '20230707'
            and service_obj_service_name = 'Auto'
            and (spd_fraud_flag = false OR spd_fraud_flag IS NULL)
            and city_name in ('Pune','Hyderabad') 
           )
    ), 

    rider_pings as

    (
        select yyyymmdd,data_riderId,data_orderid,
            count(distinct case when event_type = 'rider_busy' then unique_ping_level_id end) as riderBusy_pings,
            count(distinct case when event_type = 'rider_rejected' then unique_ping_level_id end) as riderreject_Pings,
            count(distinct case when event_type = 'rider_accepted' then unique_ping_level_id end) as accepted_pings,
            count(distinct case when event_type = 'order_already_accepted' then unique_ping_level_id end) as already_accepted_pings,
            count(distinct case when event_type = 'rider_accept_failed' then unique_ping_level_id end) as rider_accept_failure_pings,        
            count(distinct case when event_type = 'rider_acknowledged' then unique_ping_level_id end) as rider_ack_pings
        from
        (
               select yyyymmdd,quarter_hour,hh,epoch,data_propagationBatchId,data_riderId,data_orderid,
                pickup_location_hex_8,event_type,unique_ping_level_id

                from
                (
                    select propagation_batch_id as data_propagationBatchId,rider_id as data_riderId,order_id as data_orderid,yyyymmdd,quarter_hour,hh,
                    concat(propagation_batch_id,rider_id,order_id) as unique_ping_level_id,pickup_location_hex_8,
                    updated_epoch as epoch,event_type
                    from orders.dispatch_propagation_immutable
                    where yyyymmdd = cast({dt} as varchar)
                    --where yyyymmdd >= '20230620' and yyyymmdd <= '20230707'
                    and order_id in (select distinct order_id from order_filtering)
                    and event_type in ('rider_acknowledged')
                    and propagation_type in ('multicast')

                    union all

                    select data_propagationBatchId,data_riderId,data_orderid,yyyymmdd,quarter_hour,hh,
                    concat(data_propagationBatchId,data_riderId,data_orderid) as unique_ping_level_id,pickup_location_hex_8,
                    updated_epoch as epoch,event_type


                    from(
                            select event_type as eventtype,propagation_batch_id as data_propagationBatchId,rider_id as data_riderId,order_id as data_orderid,yyyymmdd,quarter_hour,hh,
                            updated_epoch,event_type,pickup_location_hex_8,drop_location_hex_8,epoch,
                            row_number() over(partition by propagation_batch_id,rider_id,order_id order by epoch desc) as row

                            from orders.dispatch_propagation_immutable
                                where yyyymmdd = cast({dt} as varchar)
                                --where yyyymmdd >= '20230620' and yyyymmdd <= '20230707'
                                and order_id in (select distinct order_id from order_filtering)
                                and propagation_type in ('multicast')
                                and event_type in ('rider_busy','order_already_accepted','rider_accept_failed','rider_rejected','rider_accepted','rider_ineligible','rider_unreachable')
                        ) base

                        where row =1
                    )
                )


                group by 1,2,3
    ),
    
   seg as  (
           select city_name,captain_id, yyyymmdd, performance_segment, consistency,concat(performance_segment,'_',consistency) as pc_segment 
            from datasets_internal.captain_auto_pc_segments_v1 
            where yyyymmdd='20230710' --cast({dt} as varchar)
            and city_name in ('Pune','Hyderabad')
    )

    -- select city_name,yyyymmdd,test_ctrl,time_period,marketplace_segment,pc_segment,data_riderId,LM_bucket,
     --       gross_orders,
     --       rider_busy_pings,
     --       rider_rejeted_pings,
     --       accepted_pings,
     --       all_accepted_pings,
     --       already_accepted_pings,
     --       total_pings,
     --       (cast(all_accepted_pings as double)/total_pings*1.00) as APR

      --  from
      --  (
    select city_name,yyyymmdd,test_ctrl,time_period,test_hexs_exposure,marketplace_segment,pc_segment,LM_bucket,

            sum(riderBusy_pings) as rider_busy_pings,
            sum(riderreject_Pings) as rider_rejeted_pings,
            sum(accepted_pings) as accepted_pings,
            sum(accepted_pings+rider_accept_failure_pings) as all_accepted_pings,
            sum(already_accepted_pings) as already_accepted_pings,
            sum(rider_ack_pings) as total_pings

            from
            (
            
            select t1.city_name,
                    t1.yyyymmdd,
                    time_period,
                    marketplace_segment,
                    t1.order_id,
                    data_riderId,
                    pc_segment,
                    LM_bucket,
                    test_hexs_exposure,
                    test_ctrl,
                    pickup_location_hex_8,
                    riderBusy_pings,
                    riderreject_Pings,
                    accepted_pings,
                    already_accepted_pings,
                    rider_accept_failure_pings,        
                    rider_ack_pings
                    


                from order_filtering as t1 

                left join rider_pings as t3 

                on t1.yyyymmdd=t3.yyyymmdd and t1.order_id = t3.data_orderid

                left join (select city_name,test_ctrl, hex_id ,time_bucket
                            from experiments_internal.auto_apr_fm_reduction_exp_230704 where city_name in ('Pune','Hyderabad')  ) hex_data

                on t1.pickup_location_hex_8=hex_data.hex_id and t1.time_period = hex_data.time_bucket 
           
               left join seg
               on t3.data_riderId=seg.captain_id
               
               
               left join (
                       select city,service,uid,segment as marketplace_segment
                        from experiments_internal.holdout_areas_v2
                        where service='Auto' and city in ('Hyderabad','Pune')
               
               ) as v1 
               on concat(t1.pickup_location_hex_8,'-',t1.time_period) = v1.uid
               
               left join (
                   select * from
                   hive.experiments.captain_exposure_to_experiment
               ) as exp
               on t3.data_riderId=exp.captain_id
                
                
                where     concat(time_period,pickup_location_hex_8) not in 
                    ('evening_peak8860b19693fffff','evening_peak8860a24b45fffff','evening_peak8860b194d9fffff',
                    'evening_peak8860a24b47fffff','evening_peak8860a24b01fffff','evening_peak8860b19695fffff',
                    'evening_peak8860b19697fffff','evening_peak8860a24b21fffff','evening_peak8860a24b2bfffff',
                    'evening_peak8860a24b63fffff','evening_peak8860b194dbfffff','evening_peak8860a24b29fffff',
                    'evening_peak8860a24b09fffff','evening_peak8860a24b69fffff','evening_peak8860a24b23fffff',
                    'evening_peak8860a24b05fffff','evening_peak8860b196bbfffff','evening_peak8860a24b2dfffff',
                    'evening_peak8860a24b65fffff','evening_peak8860b19691fffff','evening_peak8860a24b67fffff',
                    'evening_peak8860a24b6dfffff','evening_peak8860b196b3fffff','evening_peak8860b196b7fffff',
                    'evening_peak8860a24b0dfffff','evening_peak8860b196b1fffff','evening_peak8860a24b6bfffff','evening_peak8860a24b61fffff'
                    )
                    
                    and 
                    
                    case when t1.yyyymmdd >='20230710' and concat(pickup_location_hex_8,time_period) 
               in ('8860a25829fffffevening_peak','8860a259c1fffffevening_peak',
                    '8860a2583bfffffevening_peak','8860a2598dfffffevening_peak',
                    '8860a259cdfffffevening_peak','8860a25825fffffevening_peak',
                    '8860a259d5fffffevening_peak','8860a25839fffffevening_peak',
                    '8860a25821fffffevening_peak','8860a25989fffffevening_peak',
                    '8860a2583dfffffevening_peak','8860a259e1fffffevening_peak',
                    '8860a259ddfffffevening_peak','8860a25833fffffevening_peak',
                    '8860a25827fffffevening_peak','8860a259c7fffffevening_peak',
                    '8860a259c9fffffevening_peak','8860a25835fffffevening_peak',
                    '8860a25917fffffevening_peak','8860a259d1fffffevening_peak',
                    '8860a2591bfffffevening_peak','8860a259e3fffffevening_peak',
                    '8860a25831fffffevening_peak','8860a25913fffffevening_peak',
                    '8860a25807fffffevening_peak','8860a259e9fffffevening_peak',
                    '8860a2582bfffffevening_peak','8860a25823fffffevening_peak',
                    '8860a25837fffffevening_peak','8860a259edfffffevening_peak',
                    '8860a259c3fffffevening_peak','8860a25911fffffevening_peak',
                    '8860a259cbfffffevening_peak','8860a259ebfffffevening_peak',
                    '8860a25919fffffevening_peak','8860a259d9fffffevening_peak',
                    '8860a259c5fffffevening_peak','8860b19693fffffevening_peak','8860a24b45fffffevening_peak','8860b194d9fffffevening_peak',
                    '8860a24b47fffffevening_peak','8860a24b01fffffevening_peak','8860b19695fffffevening_peak','8860b19697fffffevening_peak',
                    '8860a24b21fffffevening_peak','8860a24b2bfffffevening_peak','8860a24b63fffffevening_peak','8860b194dbfffffevening_peak',
                    '8860a24b29fffffevening_peak','8860a24b09fffffevening_peak','8860a24b69fffffevening_peak','8860a24b23fffffevening_peak',
                    '8860a24b05fffffevening_peak','8860b196bbfffffevening_peak','8860a24b2dfffffevening_peak','8860a24b65fffffevening_peak',
                    '8860b19691fffffevening_peak','8860a24b67fffffevening_peak','8860a24b6dfffffevening_peak','8860b196b3fffffevening_peak',
                    '8860b196b7fffffevening_peak','8860a24b0dfffffevening_peak','8860b196b1fffffevening_peak','8860a24b6bfffffevening_peak',
                    '8860a24b61fffffevening_peak','8860a24a23fffffmorning_peak','8860a24a05fffffmorning_peak','8860a24b13fffffmorning_peak',
                    '8860a24845fffffmorning_peak','8860a24a2bfffffmorning_peak','8860a24a0dfffffmorning_peak','8860a24b55fffffmorning_peak',
                    '8860a24b11fffffmorning_peak','8860a24a6dfffffmorning_peak','8860a24b0bfffffmorning_peak','8860a259b3fffffmorning_peak',
                    '8860a24b03fffffmorning_peak','8860a259b7fffffmorning_peak','8860a24841fffffmorning_peak','8860a24a27fffffmorning_peak',
                    '8860a24a21fffffmorning_peak','8860a24849fffffmorning_peak','8860a24a63fffffmorning_peak','8860a24b15fffffmorning_peak',
                    '8860a24a69fffffmorning_peak','8860a24b1dfffffmorning_peak','8860a24b51fffffmorning_peak','8860a24a61fffffmorning_peak',
                    '8860a24a67fffffmorning_peak','8860a25993fffffmorning_peak','8860a24869fffffmorning_peak','8860a24b59fffffmorning_peak',
                    '8860a24a2dfffffmorning_peak','8860a24a6bfffffmorning_peak','8860a24a25fffffmorning_peak','8860a24b1bfffffmorning_peak',
                    '8860a24b19fffffmorning_peak','8860a2484dfffffmorning_peak','8860a24a29fffffmorning_peak','8860a24b57fffffmorning_peak',
                    '8860a24b5dfffffmorning_peak','8860a24b53fffffmorning_peak','8860a24b17fffffmorning_peak','8860a24b5bfffffmorning_peak',
                    '8860a24a65fffffmorning_peak'
                    
                    ) then 1 else 0 end =0

              )    ba               

            --where test_ctrl!=''
            group by 1,2,3,4,5,6,7,8  

            --)

            --order by 1,2,4,3 
    """.format(dt=i.strftime('%Y%m%d'))

    df_test = pd.read_sql_query(query, metabase_connection)

    print(i)
    dt1=dt1.append(df_test)


2023-07-04 00:00:00
2023-07-05 00:00:00
2023-07-06 00:00:00
2023-07-07 00:00:00


In [74]:
dt1#""[dt1.data_riderId=='610e1ca27c08d4479fb59e55']

NameError: name 'dt1' is not defined

In [91]:
import copy

In [145]:
#new_one=copy.deepcopy(dt1)

In [148]:
total_dt=pd.concat([new_one,dt1]).drop_duplicates()

In [149]:
total_dt=total_dt.reset_index(drop=True)

In [151]:
total_dt.to_csv('data_of_exposure_raw.csv',index=False)

In [153]:
total_dt

,city_name,yyyymmdd,test_ctrl,time_period,test_hexs_exposure,marketplace_segment,pc_segment,LM_bucket,rider_busy_pings,rider_rejeted_pings,accepted_pings,all_accepted_pings,already_accepted_pings,total_pings
0,Hyderabad,20230708,ctrl_3,evening_peak,low_exposure,in-high,LP_inter,2-5,1528.0,1431.0,25.0,25.0,311.0,3458.0
1,Hyderabad,20230708,ctrl_3,afternoon,high_exposure,balanced-low,HP_Daily,5-8,96.0,126.0,46.0,60.0,260.0,558.0
2,Hyderabad,20230708,test_2,afternoon,high_exposure,balanced-high,MP_Daily,8-12,253.0,266.0,86.0,117.0,548.0,1222.0
3,Hyderabad,20230708,ctrl_3,morning_peak,high_exposure,in-low,MP_intra,8-12,1.0,6.0,11.0,18.0,153.0,180.0
4,Pune,20230708,test_2,afternoon,high_exposure,out-low,MP_Daily,5-8,32.0,22.0,8.0,8.0,31.0,95.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306505,Hyderabad,20230707,ctrl_1,evening_peak,None,in-low,LP_inter,2-5,0.0,1.0,0.0,0.0,3.0,4.0
306506,Pune,20230707,ctrl_4,afternoon,high_exposure,balanced-low,UHP_intra,>12,0.0,0.0,0.0,0.0,1.0,1.0
306507,Pune,20230707,test_2,afternoon,None,in-low,None,8-12,NaN,NaN,NaN,NaN,NaN,NaN
306508,Pune,20230707,ctrl_3,morning_peak,low_exposure,balanced-low,MP_inter,>12,0.0,0.0,0.0,0.0,1.0,1.0


In [154]:
for_summary=total_dt.groupby(['city_name','test_ctrl','test_hexs_exposure'],as_index=False)[[
    'rider_busy_pings', 'rider_rejeted_pings',
    'accepted_pings', 'all_accepted_pings', 'already_accepted_pings','total_pings'
]].sum()

In [155]:
for_summary['APR']=for_summary.all_accepted_pings/for_summary.total_pings

In [156]:
for_summary['busy_pings_%']=for_summary.rider_busy_pings/for_summary.total_pings

In [157]:
for_summary['reject_pings_%']=for_summary.rider_rejeted_pings/for_summary.total_pings

In [158]:
for_summary.to_clipboard()

In [104]:
total_dt.columns

Index(['city_name', 'yyyymmdd', 'test_ctrl', 'time_period',
       'test_hexs_exposure', 'marketplace_segment', 'pc_segment', 'LM_bucket',
       'gross_orders', 'rider_busy_pings', 'rider_rejeted_pings',
       'accepted_pings', 'all_accepted_pings', 'already_accepted_pings',
       'total_pings'],
      dtype='object')